In [1]:
pip install reportlab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 8.4 MB/s eta 0:00:00


In [2]:
pip install cairosvg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 781.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 678.6 kB/s eta 0:00:00


In [12]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import re
import os
from google.colab import files
from reportlab.lib.pagesizes import letter
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import SimpleDocTemplate, Paragraph, Table, TableStyle, Image, Spacer, PageBreak, ListFlowable, ListItem
from reportlab.lib.units import inch  # Import inch for units
from math import pi
from google.colab import drive
import matplotlib.pyplot as plt
from PIL import Image as PILImage, ImageOps, ImageEnhance, ImageDraw
from sklearn.linear_model import LinearRegression
import requests
from io import BytesIO
from reportlab.pdfgen import canvas
from reportlab.lib.enums import TA_CENTER
import matplotlib.patches as patches
from matplotlib.patches import Circle
import shutil
from IPython.display import FileLink
import cairosvg

In [13]:
# Mount your Google Drive
drive.mount('/content/drive')

# Define the path to your CSV file in Google Drive
file_path = '/content/drive/MyDrive/CFB_Model/original_2024_projections.csv'

uploaded = files.upload()

# Load the CSV file into a DataFrame
df_projections = pd.read_csv(file_path)
# Load the CSV file with team information
team_data = pd.read_csv('/content/drive/MyDrive/CFB_Model/Raw Data/2024_team_info.csv')
# Preview the data to ensure it's loaded correctly
df_projections.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Saving Logo_of_college_football_playoff.svg.png to Logo_of_college_football_playoff.svg.png


,Year,Week,Home Team,Away Team,Avg_Home_Score,Avg_Away_Score,%Win Home,%Win Away,Predicted Spread,Over_Under
0,2024,0,Georgia Tech,Florida State,18.983,35.685,0.00,1.00,Florida State - 16.70,O/U: 54.67
1,2024,0,Nevada,SMU,15.969,43.187,0.00,1.00,SMU - 27.22,O/U: 59.16
2,2024,1,Alabama,Western Kentucky,47.165,15.000,1.00,0.00,Alabama - 32.17,O/U: 62.17
3,2024,1,Arizona,New Mexico,46.818,23.742,1.00,0.00,Arizona - 23.08,O/U: 70.56
4,2024,1,Arizona State,Wyoming,29.659,26.965,0.74,0.26,Arizona State - 2.69,O/U: 56.62


In [5]:
# Add new columns for tracking actual scores, team performance, and Vegas data
df_projections['Actual_Home_Score'] = pd.NA
df_projections['Actual_Away_Score'] = pd.NA
df_projections['Home_Offensive_Performance'] = pd.NA
df_projections['Away_Offensive_Performance'] = pd.NA
df_projections['Home_Defensive_Performance'] = pd.NA
df_projections['Away_Defensive_Performance'] = pd.NA
df_projections['Vegas_Spread'] = pd.NA
df_projections['Vegas_Total'] = pd.NA

# Preview the updated dataframe
df_projections.head()

,Year,Week,Home Team,Away Team,Avg_Home_Score,Avg_Away_Score,%Win Home,%Win Away,Predicted Spread,Over_Under,Actual_Home_Score,Actual_Away_Score,Home_Offensive_Performance,Away_Offensive_Performance,Home_Defensive_Performance,Away_Defensive_Performance,Vegas_Spread,Vegas_Total
0,2024,0,Georgia Tech,Florida State,18.983,35.685,0.00,1.00,Florida State - 16.70,O/U: 54.67,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,2024,0,Nevada,SMU,15.969,43.187,0.00,1.00,SMU - 27.22,O/U: 59.16,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,2024,1,Alabama,Western Kentucky,47.165,15.000,1.00,0.00,Alabama - 32.17,O/U: 62.17,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,2024,1,Arizona,New Mexico,46.818,23.742,1.00,0.00,Arizona - 23.08,O/U: 70.56,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,2024,1,Arizona State,Wyoming,29.659,26.965,0.74,0.26,Arizona State - 2.69,O/U: 56.62,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [25]:
# Function to calculate the performance for a specific team up to a given week
def calculate_team_performance(df, team, week):
    # Filter the dataframe to get all games for the team up to the given week
    past_games = df[((df['Home Team'] == team) | (df['Away Team'] == team)) & (df['Week'] <= week)]

    if past_games.empty:
        return None, None, None  # Return None values if no past games are found

    # Calculate total offensive and defensive performance metrics for games the team played
    offensive_performance = []
    defensive_performance = []

    for index, game in past_games.iterrows():
        if game['Home Team'] == team:
            # Team is playing at home
            offensive_performance.append(game['Home_Offensive_Performance'])
            defensive_performance.append(game['Home_Defensive_Performance'])
        elif game['Away Team'] == team:
            # Team is playing away
            offensive_performance.append(game['Away_Offensive_Performance'])
            defensive_performance.append(game['Away_Defensive_Performance'])

    # Calculate average offensive and defensive performance
    avg_offensive_performance = sum(offensive_performance) / len(offensive_performance) if offensive_performance else 0
    avg_defensive_performance = sum(defensive_performance) / len(defensive_performance) if defensive_performance else 0

    return past_games, avg_offensive_performance, avg_defensive_performance

# Function to plot the team's performance over past games
def plot_team_performance(team, past_games):
    # Extract the weeks, offensive, and defensive performance
    weeks = past_games['Week']
    if team == past_games.iloc[0]['Home Team']:
        offensive_perf = past_games['Home_Offensive_Performance']
        defensive_perf = past_games['Home_Defensive_Performance']
    else:
        offensive_perf = past_games['Away_Offensive_Performance']
        defensive_perf = past_games['Away_Defensive_Performance']

    # Create the plot
    plt.figure(figsize=(10, 6))

    # Plot offensive performance
    plt.plot(weeks, offensive_perf, label=f'{team} Offensive Performance', marker='o')

    # Plot defensive performance
    plt.plot(weeks, defensive_perf, label=f'{team} Defensive Performance', marker='o')

    # Add labels and title
    plt.xlabel('Week')
    plt.ylabel('Performance Ratio')
    plt.title(f'{team} Performance Over Time')
    plt.axhline(1, color='gray', linestyle='--', label='Expected Performance')  # Baseline for expected performance

    # Add legend
    plt.legend()

    # Show the plot
    plt.show()

# Game processing section where you visualize performance
def input_vegas_spread(df, week):
    # Filter for the games of the specified week
    week_games = df[df['Week'] == week]

    if week_games.empty:
        print(f"No games found for Week {week}.")
        return

    for index, game in week_games.iterrows():
        home_team = game['Home Team']
        away_team = game['Away Team']
        print(f"\nGame: {home_team} vs. {away_team}")

        # Prompt the user to enter the Vegas spread and total
        vegas_spread = input(f"Enter Vegas spread (e.g., {home_team} -5): ")
        vegas_total = input("Enter Vegas total points (e.g., 54.5): ")

        # Store the entered data
        df.at[index, 'Vegas_Spread'] = vegas_spread
        df.at[index, 'Vegas_Total'] = vegas_total

# Ensure actual score columns exist and are of correct type
def initialize_actual_score_columns(df):
    if 'Actual_Home_Score' not in df.columns:
        df['Actual_Home_Score'] = pd.NA  # Initialize as missing
    if 'Actual_Away_Score' not in df.columns:
        df['Actual_Away_Score'] = pd.NA  # Initialize as missing

    # Ensure performance columns exist
    performance_columns = [
        'Home_Offensive_Performance', 'Away_Offensive_Performance',
        'Home_Defensive_Performance', 'Away_Defensive_Performance'
    ]

    for col in performance_columns:
        if col not in df.columns:
            df[col] = pd.NA  # Initialize as missing

    # Convert to float type if necessary (this should prevent the warning)
    df['Actual_Home_Score'] = pd.to_numeric(df['Actual_Home_Score'], errors='coerce')
    df['Actual_Away_Score'] = pd.to_numeric(df['Actual_Away_Score'], errors='coerce')

# Function to calculate performance metrics
def calculate_performance_metrics(row):
    # Calculate Home Offensive and Away Defensive performance based on the home team's score
    if pd.notna(row['Actual_Home_Score']) and row['Avg_Home_Score'] > 0:
        row['Home_Offensive_Performance'] = row['Actual_Home_Score'] / row['Avg_Home_Score']
        row['Away_Defensive_Performance'] = row['Actual_Home_Score'] / row['Avg_Home_Score']

    # Calculate Away Offensive and Home Defensive performance based on the away team's score
    if pd.notna(row['Actual_Away_Score']) and row['Avg_Away_Score'] > 0:
        row['Away_Offensive_Performance'] = row['Actual_Away_Score'] / row['Avg_Away_Score']
        row['Home_Defensive_Performance'] = row['Actual_Away_Score'] / row['Avg_Away_Score']

    return row

# Function to input actual scores for each game in the selected week
def input_actual_scores(df, week):
    # Initialize the actual score columns and performance columns if they don't exist
    initialize_actual_score_columns(df)

    # Filter the dataframe for the selected week
    week_games = df[df['Week'] == week]

    if week_games.empty:
        print(f"No games found for Week {week}.")
        return

    print(f"\nEntering actual scores for Week {week}\n")

    # Loop through each game and prompt the user to enter the actual scores
    for index, row in week_games.iterrows():
        home_team = row['Home Team']
        away_team = row['Away Team']

        print(f"\nGame: {home_team} vs. {away_team}")

        # Input the actual scores for home and away teams, cast to float
        actual_home_score = float(input(f"Enter actual score for {home_team}: "))
        actual_away_score = float(input(f"Enter actual score for {away_team}: "))

        # Store the entered data
        df.at[index, 'Actual_Home_Score'] = actual_home_score
        df.at[index, 'Actual_Away_Score'] = actual_away_score

        # Calculate and store performance metrics
        df.iloc[index] = calculate_performance_metrics(df.iloc[index])

    print(f"\nData entry completed for Week {week}")

def parse_spread(spread_str):
    """
    Parse the spread string into a numeric value and the favored team.
    Example: 'Georgia - 13.5' -> (-13.5, 'Georgia')
    """
    try:
        # Strip any leading/trailing whitespace
        spread_str = spread_str.strip()

        # Regular expression to match the format 'Team - Spread'
        match = re.match(r'^(.*?)\s*-\s*([0-9\.]+)$', spread_str)

        if match:
            favored_team = match.group(1).strip()
            spread = float(match.group(2).strip())
            return spread, favored_team
        else:
            print(f"Invalid format for spread '{spread_str}'")
            return None, None
    except (ValueError, IndexError) as e:
        print(f"Failed to parse spread '{spread_str}': {e}")
        return None, None

def analyze_weekly_betting_performance(df, max_week):
    performance_data = []
    overall_data = {'total_games': 0, 'correct_predictions': 0}

    for week in range(max_week):
        week_data = df[df['Week'] == week]
        total_games = len(week_data)
        correct_predictions = 0

        for _, row in week_data.iterrows():
            home_team = row['Home Team']
            away_team = row['Away Team']

            # Parse spreads
            vegas_spread, vegas_favored_team = parse_spread(row['Vegas_Spread'])
            predicted_spread, predicted_favored_team = parse_spread(row['Predicted Spread'])

            # Skip rows with invalid spreads
            if vegas_spread is None or predicted_spread is None:
                continue

            # Calculate actual score differences
            home_score = row['Actual_Home_Score']
            away_score = row['Actual_Away_Score']

            # Determine actual spread based on actual scores
            vegas_favored_team_score = home_score if vegas_favored_team == home_team else away_score
            vegas_other_team_score = away_score if vegas_favored_team == home_team else home_score
            actual_spread = vegas_other_team_score - vegas_favored_team_score

            # Check if Vegas favored team covered the spread
            favored_team_covered = actual_spread < vegas_spread

            # Check if the model predicted the correct team and spread coverage
            if vegas_favored_team == predicted_favored_team:
                # Both Vegas and the model favor the same team
                if predicted_spread < vegas_spread and favored_team_covered:
                    correct_predictions += 1
                elif predicted_spread > vegas_spread and not favored_team_covered:
                    correct_predictions += 1
            elif predicted_spread > vegas_spread:
                correct_predictions += 1

        week_accuracy = (correct_predictions / total_games) * 100 if total_games > 0 else 0
        performance_data.append({
            'Week': week,
            'Total Games': total_games,
            'Correct Predictions': correct_predictions,
            'Accuracy': week_accuracy
        })

        # Update overall performance
        overall_data['total_games'] += total_games
        overall_data['correct_predictions'] += correct_predictions

    # Calculate overall accuracy
    overall_accuracy = (overall_data['correct_predictions'] / overall_data['total_games']) * 100 if overall_data['total_games'] > 0 else 0

    return performance_data, overall_data, overall_accuracy

def gather_team_performance(df, team, week):
    """
    Gathers performance data for a given team up to the specified week.
    Returns weeks, offensive performance, and defensive performance lists.
    """
    # Check the structure of the DataFrame
    #print(f"Data columns: {df.columns}")

    # Filter for games where the team was involved (check for 'Home Team' and 'Away Team' being correct column names)
    team_home_games = df[(df['Home Team'] == team) & (df['Week'] < week)]
    team_away_games = df[(df['Away Team'] == team) & (df['Week'] < week)]

    # Print filtered data for debugging
    #print(f"Team: {team}, Home Games Filtered: {team_home_games.shape[0]}, Away Games Filtered: {team_away_games.shape[0]}")

    # Initialize lists to store performance data
    offensive_performance = []
    defensive_performance = []
    weeks = []

    # Collect data from home games
    for _, row in team_home_games.iterrows():
        offensive_performance.append(row['Home_Offensive_Performance'])
        defensive_performance.append(row['Home_Defensive_Performance'])
        weeks.append(row['Week'])

    # Collect data from away games
    for _, row in team_away_games.iterrows():
        offensive_performance.append(row['Away_Offensive_Performance'])
        defensive_performance.append(row['Away_Defensive_Performance'])
        weeks.append(row['Week'])

    # Print the collected data
    #print(f"Team: {team}, Weeks: {weeks}, Offensive: {offensive_performance}, Defensive: {defensive_performance}")

    return weeks, offensive_performance, defensive_performance

# Function to calculate control limits
def calculate_control_limits(performance_data):
    # Remove NaN values for control limit calculation
    clean_data = [val for val in performance_data if not np.isnan(val)]
    if len(clean_data) > 0:
        avg_performance = np.mean(clean_data)
        std_dev = np.std(clean_data)
        UCL = avg_performance + 3 * std_dev
        LCL = avg_performance - 3 * std_dev
    else:
        avg_performance = UCL = LCL = 1  # Default if no data is available
    return avg_performance, UCL, LCL

def generate_team_performance_graphs(df, home_info, away_info, current_week):
    performance_graphs = {}

    # Home team information
    home_team = home_info['Team']
    home_team_color = home_info['Color']
    home_team_logo_url = home_info['Logos[0]']

    # Get the weeks and performance data for the home team
    weeks, offensive_perf_home, defensive_perf_home = gather_team_performance(df, home_team, current_week)

    # Plot home team's vertical bar chart with average line
    home_chart = plot_vertical_bar_chart_with_avg(
        weeks=weeks,
        offensive_perf=offensive_perf_home,
        defensive_perf=defensive_perf_home,
        team_name=home_team,
        team_color=home_team_color,
        team_logo_url=home_team_logo_url,
        max_week=current_week  # Correctly passing max_week as an integer
    )

    performance_graphs['home_combined'] = home_chart

    # Away team information
    away_team = away_info['Team']
    away_team_color = away_info['Color']
    away_team_logo_url = away_info['Logos[0]']

    # Get the weeks and performance data for the away team
    weeks, offensive_perf_away, defensive_perf_away = gather_team_performance(df, away_team, current_week)

    # Plot away team's vertical bar chart with average line
    away_chart = plot_vertical_bar_chart_with_avg(
        weeks=weeks,
        offensive_perf=offensive_perf_away,
        defensive_perf=defensive_perf_away,
        team_name=away_team,
        team_color=away_team_color,
        team_logo_url=away_team_logo_url,
        max_week=current_week  # Correctly passing max_week as an integer
    )

    performance_graphs['away_combined'] = away_chart

    return performance_graphs

# Function to generate a separate legend as an image
def generate_combined_legend():
    fig, ax = plt.subplots(figsize=(2, 1))  # Create a dummy figure to hold the legend
    fig.patch.set_visible(False)  # Hide the figure background
    ax.axis('off')  # Hide axes

    # Create fake plots for the legend
    offensive_line = plt.Line2D([0], [0], color='blue', marker='o', label='Offensive Performance')
    defensive_line = plt.Line2D([0], [0], color='blue', marker='o', label='Defensive Performance')
    avg_line = plt.Line2D([0], [0], color='green', linestyle='--', label='Average (CL)')
    ucl_line = plt.Line2D([0], [0], color='red', linestyle='--', label='Upper Control Limit (UCL)')
    lcl_line = plt.Line2D([0], [0], color='red', linestyle='--', label='Lower Control Limit (LCL)')

    # Create the legend
    ax.legend(handles=[offensive_line, defensive_line, avg_line, ucl_line, lcl_line], loc='center', frameon=False)

    # Save the legend as an image
    legend_image = 'legend_performance.png'
    plt.savefig(legend_image, bbox_inches='tight', dpi=300)
    plt.close()

    return legend_image

################################################################################

def moving_average(data, window_size=3):
    """ Calculate the moving average of the data, handling missing values. """
    # Remove NaN values from the data
    clean_data = [x for x in data if not np.isnan(x)]

    # Check if we have enough data points to calculate the moving average
    if len(clean_data) >= window_size:
        return np.convolve(clean_data, np.ones(window_size) / window_size, mode='valid')
    else:
        # Return the clean data directly if insufficient points for moving average
        return clean_data

def plot_combined_performance_chart(weeks, offensive_perf, defensive_perf, team_name, team_color, team_logo_url):
    """
    Plots combined offensive and defensive performance with moving average trendlines, ensuring tight layout for smaller figure size.
    """
    # Calculate moving averages, only if there's enough data
    offensive_ma = moving_average(offensive_perf, window_size=3)
    defensive_ma = moving_average(defensive_perf, window_size=3)

    # Set exact figure size to match the Image dimensions (350x250)
    fig, ax = plt.subplots(figsize=(3.5, 2.5))  # 3.5 inches wide, 2.5 inches tall

    # Plot offensive and defensive performance
    ax.plot(weeks, offensive_perf, label='Offensive Performance', marker='o', color=team_color, linestyle='-', alpha=0.6)
    ax.plot(weeks, defensive_perf, label='Defensive Performance', marker='o', color='grey', linestyle='-', alpha=0.6)

    # Plot moving average trendlines only if we have enough data
    if len(offensive_ma) > 0:
        ax.plot(weeks[:len(offensive_ma)], offensive_ma, label='Offensive MA (3 weeks)', color=team_color, linestyle='--', lw=1.5)
    if len(defensive_ma) > 0:
        ax.plot(weeks[:len(defensive_ma)], defensive_ma, label='Defensive MA (3 weeks)', color='black', linestyle='--', lw=1.5)

    # Add title and labels with reduced font size
    ax.set_title(f'{team_name} Performance', fontsize=10)
    ax.set_xlabel('Week', fontsize=8)
    ax.set_ylabel('Performance', fontsize=8)

    # Adjust tick labels to be smaller
    ax.set_xticks(weeks)
    ax.set_xticklabels(weeks, fontsize=7, rotation=45)
    ax.tick_params(axis='y', labelsize=7)

    # Add team logo to the plot as a watermark, ensuring it fits the smaller size
    try:
        logo = PILImage.open(BytesIO(requests.get(team_logo_url).content))
        ax.imshow(logo, extent=[len(weeks)-3, len(weeks), -1, 1], aspect='auto', alpha=0.1)  # Watermark logo with reduced alpha
    except:
        print(f"Error loading logo for {team_name}")

    # Ensure grid and legend are small and don't take too much space
    ax.grid(True)
    ax.legend(fontsize=6, loc='upper left')  # Smaller legend size and position to fit

    # Use tight layout to ensure no overlap
    plt.tight_layout(pad=1.0)  # Adjust padding if needed

    # Save the chart
    chart_filename = f'{team_name}_combined_performance_chart.png'
    plt.savefig(chart_filename, bbox_inches='tight', dpi=150)
    plt.close()

    return chart_filename


def plot_side_by_side_bar_chart(weeks, offensive_perf, defensive_perf, team_name, team_color, team_logo_url):
    """
    Plots a side-by-side bar chart comparing offensive and defensive performance for each week.
    Handles missing data by ensuring weeks, offensive_perf, and defensive_perf have matching shapes.
    Adjusts for a compact format (250x300 size).
    """
    # Ensure the lengths of the data arrays are the same
    min_length = min(len(weeks), len(offensive_perf), len(defensive_perf))
    weeks = weeks[:min_length]
    offensive_perf = offensive_perf[:min_length]
    defensive_perf = defensive_perf[:min_length]

    # Handle missing values: replace with zeros or np.nan as per preference
    offensive_perf = [0 if np.isnan(x) else x for x in offensive_perf]
    defensive_perf = [0 if np.isnan(x) else x for x in defensive_perf]

    # Set exact figure size for the Image dimensions (250x300)
    fig, ax = plt.subplots(figsize=(2.5, 3.0))  # 2.5 inches wide, 3.0 inches tall

    # Define bar width and x-axis positions
    bar_width = 0.35
    indices = np.arange(len(weeks))  # X-axis positions

    # Plot offensive and defensive performance as side-by-side bars
    ax.bar(indices - bar_width/2, offensive_perf, bar_width, label='Offensive', color=team_color, alpha=0.7)
    ax.bar(indices + bar_width/2, defensive_perf, bar_width, label='Defensive', color='grey', alpha=0.7)

    # Add title and labels with reduced font size
    ax.set_title(f'{team_name} Performance', fontsize=10)
    ax.set_xlabel('Week', fontsize=8)
    ax.set_ylabel('Performance', fontsize=8)

    # Set x-ticks to be the week numbers, ensuring they fit
    ax.set_xticks(indices)
    ax.set_xticklabels(weeks, fontsize=7, rotation=45)

    # Adjust y-axis tick size
    ax.tick_params(axis='y', labelsize=7)

    # Add the team logo to the plot as a watermark, ensuring it fits the smaller size
    try:
        logo = PILImage.open(BytesIO(requests.get(team_logo_url).content))
        ax.imshow(logo, extent=[len(weeks)-3, len(weeks), -1, 1], aspect='auto', alpha=0.1)  # Watermark logo
    except:
        print(f"Error loading logo for {team_name}")

    # Add a grid for better readability
    ax.grid(True, which='both', axis='y', linestyle='--', alpha=0.5)

    # Add a smaller legend
    ax.legend(fontsize=6, loc='upper left')

    # Use tight layout to ensure no overlap
    plt.tight_layout(pad=1.0)

    # Save the chart
    chart_filename = f'{team_name}_side_by_side_performance_chart.png'
    plt.savefig(chart_filename, bbox_inches='tight', dpi=150)
    plt.close()

    return chart_filename

def plot_vertical_bar_chart_with_avg(weeks, offensive_perf, defensive_perf, team_name, team_color, team_logo_url, max_week):
    """
    Plots a vertical bar chart showing offensive and defensive performance by week,
    with a vertical line representing average offensive and defensive performance.
    Bars to the right indicate overperformance, bars to the left indicate underperformance.
    Handles cases where there are gaps in data and provides fallback if the team hasn't played.
    """
    # Ensure max_week is treated as an integer
    max_week = int(max_week)

    # Set exact figure size for the Image dimensions (250x350)
    fig, ax = plt.subplots(figsize=(2.5, 3.5))  # 2.5 inches wide, 3.5 inches tall

    # Handle missing values by replacing NaNs with zeros or flagging as no data
    if len(weeks) == 0:
        # No games played yet
        ax.text(0.5, 0.5, 'No Data Available', horizontalalignment='center', verticalalignment='center', fontsize=12, color='red')
        plt.tight_layout()
        chart_filename = f'{team_name}_vertical_bar_chart_with_avg.png'
        plt.savefig(chart_filename, bbox_inches='tight', dpi=150)
        plt.close()
        return chart_filename

    offensive_perf = [0 if np.isnan(x) else x for x in offensive_perf]
    defensive_perf = [0 if np.isnan(x) else x for x in defensive_perf]

    # Calculate average performance
    avg_offensive = np.mean(offensive_perf) if len(offensive_perf) > 0 else 0
    avg_defensive = np.mean(defensive_perf) if len(defensive_perf) > 0 else 0

    # Plot vertical bars, color-coded by over/underperformance
    bar_width = 0.4
    ax.barh(weeks, offensive_perf, bar_width, label='Offensive', color=team_color, alpha=0.7)
    ax.barh(weeks, [-def_perf for def_perf in defensive_perf], bar_width, label='Defensive', color='grey', alpha=0.7, left=0)

    # Add vertical lines for average offensive and defensive performance
    ax.axvline(avg_offensive, color=team_color, linestyle='--', label=f'Avg Offensive: {avg_offensive:.2f}')
    ax.axvline(-avg_defensive, color='grey', linestyle='--', label=f'Avg Defensive: {avg_defensive:.2f}')

    # Lock the x-axis between -2 and 2
    ax.set_xlim([-2, 2])

    # Relabel the x-axis with {2, 1, 0, 1, 2} format
    ax.set_xticks([-2, -1, 0, 1, 2])
    ax.set_xticklabels([2, 1, 0, 1, 2])

    # Set the y-axis from 0 (bottom) to max_week (top)
    ax.set_ylim([0, max_week])

    # Add title and labels with reduced font size
    ax.set_title(f'{team_name} Performance', fontsize=10)
    ax.set_xlabel('% Points vs Expected (x100)', fontsize=8)  # Updated x-axis label
    ax.set_ylabel('Week', fontsize=8)  # Updated y-axis label

    # Add the team logo to the plot as a square, locking it to the center of the chart
    try:
        logo = PILImage.open(BytesIO(requests.get(team_logo_url).content))
        # Increase the logo size by adjusting the extent
        logo_size = (max_week * 0.35)  # Larger logo proportional to chart height
        ax.imshow(logo, extent=[-logo_size, logo_size, max_week * 0.2, max_week * 0.8], aspect='auto', alpha=0.1)
    except Exception as e:
        print(f"Error loading logo for {team_name}: {e}")
        # Fallback to a square gray placeholder if the logo fails to load
        placeholder = np.full((50, 50, 3), 200, dtype=np.uint8)  # A gray square
        logo_size = (max_week * 0.35)  # Placeholder logo size
        ax.imshow(placeholder, extent=[-logo_size, logo_size, max_week * 0.2, max_week * 0.8], aspect='auto', alpha=0.1)

    # Add a grid for better readability
    ax.grid(True, which='both', axis='x', linestyle='--', alpha=0.5)

    # Move the legend below the x-axis and center it
    ax.legend(fontsize=6, loc='upper center', bbox_to_anchor=(0.5, -0.2), borderaxespad=0., ncol=2)

    # Use tight layout to ensure no overlap
    plt.tight_layout(pad=1.0)  # Adjust the padding as needed

    # Save the chart
    chart_filename = f'{team_name}_vertical_bar_chart_with_avg.png'
    plt.savefig(chart_filename, bbox_inches='tight', dpi=150)
    plt.close()

    return chart_filename

################################################################################

# Function to clamp the adjustment within the range [0.5, 2.0]
def clamp_adjustment(adjustment):
    lower_bound = 0.5
    upper_bound = 2.0
    return max(lower_bound, min(adjustment, upper_bound))

# Function to calculate the adjusted score
def calculate_adjusted_score(df, team, week, is_home):
    # Filter for games where the team has played up to the current week
    past_games = df[((df['Home Team'] == team) | (df['Away Team'] == team)) & (df['Week'] < week)]

    # Initialize performance ratios
    offensive_performance_ratios = []
    defensive_performance_ratios = []

    for _, row in past_games.iterrows():
        if row['Home Team'] == team:
            offensive_performance_ratios.append(row['Home_Offensive_Performance'])
            defensive_performance_ratios.append(row['Home_Defensive_Performance'])
        else:
            offensive_performance_ratios.append(row['Away_Offensive_Performance'])
            defensive_performance_ratios.append(row['Away_Defensive_Performance'])

    # Calculate the average performance adjustment
    avg_offensive_adjustment = sum(offensive_performance_ratios) / len(offensive_performance_ratios) if offensive_performance_ratios else 1
    avg_defensive_adjustment = sum(defensive_performance_ratios) / len(defensive_performance_ratios) if defensive_performance_ratios else 1

    # Clamp the adjustments within the range [0.5, 2.0]
    avg_offensive_adjustment = clamp_adjustment(avg_offensive_adjustment)
    avg_defensive_adjustment = clamp_adjustment(avg_defensive_adjustment)

    # Return both offensive and defensive adjustments
    return avg_offensive_adjustment, avg_defensive_adjustment

styles = getSampleStyleSheet()

# Title Page Creation
def create_title_page(elements, max_week, season_year, author_name="Noah Barulic"):
    """
    Creates the title page with logos, report title, season, week, and author's name.
    The CFP logo is used as a very faded watermark behind the entire title page.

    :param elements: List of elements to add to the report.
    :param max_week: The current week for the report.
    :param season_year: The current season year.
    :param author_name: The author's name (default: Noah Barulic).
    """
    # Define styles for the title and subtitle
    styles = getSampleStyleSheet()
    title_style = styles['Title']
    title_style.alignment = 1  # Center-align
    subtitle_style = styles['Heading2']
    subtitle_style.alignment = 1  # Center-align
    author_style = styles['Normal']
    author_style.alignment = 1  # Center-align

    # Load the local College Football Playoff logo
    try:
        playoff_logo = PILImage.open("Logo_of_college_football_playoff.svg.png")  # Reference the uploaded image

        # Convert the image to RGBA (to support transparency)
        playoff_logo = playoff_logo.convert("RGBA")

        # Create a transparent image for the faded watermark
        faded_logo = PILImage.new("RGBA", playoff_logo.size, (255, 255, 255, 0))

        # Reduce the opacity (alpha channel) of the logo (make it more transparent)
        for x in range(playoff_logo.width):
            for y in range(playoff_logo.height):
                r, g, b, a = playoff_logo.getpixel((x, y))
                faded_logo.putpixel((x, y), (r, g, b, int(a * 0.1)))  # Reduce opacity to 10%

        # Save the faded logo as a temporary image
        faded_logo_path = "faded_cfp_logo.png"
        faded_logo.save(faded_logo_path, format="PNG")

        # Create an image for the faded watermark
        watermark_image = Image(faded_logo_path, width=6 * inch, height=6 * inch)
    except Exception as e:
        print(f"Error loading logos: {e}")
        watermark_image = Spacer(1, 6 * inch)

    # Add the watermark image first, center it on the page
    elements.append(Spacer(1, 1 * inch))
    elements.append(watermark_image)

    # Add the title, season, and week info
    elements.append(Spacer(1, -5 * inch))  # Shift the content above the watermark
    elements.append(Paragraph(f"College Football Pregame Betting Analysis", title_style))
    elements.append(Paragraph(f"Predictive Betting Report", subtitle_style))
    elements.append(Spacer(1, 0.2 * inch))
    elements.append(Paragraph(f"Season: {season_year} | Week {max_week}", subtitle_style))
    elements.append(Spacer(1, 0.4 * inch))
    elements.append(Paragraph(f"Prepared by: {author_name}", author_style))

    # Add some extra spacing below
    elements.append(Spacer(1, 1 * inch))

    # Add a page break at the end to start the next section on a new page
    elements.append(PageBreak())

def create_model_performance_section(elements, performance_data, overall_data, overall_accuracy, styles):
    # Section heading
    elements.append(Paragraph("Model Performance Against the Spread", styles['Heading2']))

    # Add individual week reports
    for week_performance in performance_data:
        # Week title
        week_title = Paragraph(f"Week {week_performance['Week']} Performance", ParagraphStyle('Title', fontSize=14, spaceAfter=12))
        elements.append(week_title)

        # Improved table structure with alternate row coloring
        week_table_data = [
            ['Total Games', 'Correct Predictions', 'Accuracy'],
            [week_performance['Total Games'], week_performance['Correct Predictions'], f"{week_performance['Accuracy']:.2f}%"]
        ]

        # Styling: alternating row colors, bold headers, and better spacing
        week_table = Table(week_table_data, colWidths=[2 * inch, 2 * inch, 2 * inch])
        week_table.setStyle(TableStyle([
            ('BACKGROUND', (0, 0), (-1, 0), colors.Color(0.2, 0.4, 0.6)),  # Darker header background
            ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),  # White text for the header
            ('BACKGROUND', (0, 1), (-1, -1), colors.Color(0.95, 0.95, 0.95)),  # Light background for rows
            ('TEXTCOLOR', (0, 1), (-1, -1), colors.black),  # Black text for body rows
            ('ALIGN', (0, 0), (-1, -1), 'CENTER'),  # Center-align all cells
            ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),  # Bold header font
            ('FONTNAME', (0, 1), (-1, -1), 'Helvetica'),  # Normal font for data rows
            ('FONTSIZE', (0, 0), (-1, -1), 10),  # Set font size
            ('BOTTOMPADDING', (0, 0), (-1, 0), 12),  # Padding for the header
            ('BOTTOMPADDING', (0, 1), (-1, -1), 10),  # Padding for the data rows
            ('GRID', (0, 0), (-1, -1), 0.5, colors.grey)  # Thin grid lines
        ]))

        elements.append(week_table)
        elements.append(Spacer(1, 0.25 * inch))  # Add more space between week reports

    # Overall performance
    overall_title = Paragraph("Overall Performance", ParagraphStyle('Title', fontSize=14, spaceAfter=12))
    elements.append(overall_title)

    # Improved overall performance table
    overall_table_data = [
        ['Total Games', 'Correct Predictions', 'Overall Accuracy'],
        [overall_data['total_games'], overall_data['correct_predictions'], f"{overall_accuracy:.2f}%"]
    ]

    overall_table = Table(overall_table_data, colWidths=[2 * inch, 2 * inch, 2 * inch])
    overall_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.Color(0.2, 0.4, 0.6)),  # Darker header background
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),  # White text for the header
        ('BACKGROUND', (0, 1), (-1, -1), colors.Color(0.9, 0.9, 0.9)),  # Slightly different background for overall
        ('TEXTCOLOR', (0, 1), (-1, -1), colors.black),  # Black text for body rows
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),  # Center-align all cells
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),  # Bold header font
        ('FONTNAME', (0, 1), (-1, -1), 'Helvetica'),  # Normal font for data rows
        ('FONTSIZE', (0, 0), (-1, -1), 10),  # Set font size
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),  # Padding for the header
        ('BOTTOMPADDING', (0, 1), (-1, -1), 10),  # Padding for the data rows
        ('GRID', (0, 0), (-1, -1), 0.5, colors.grey)  # Thin grid lines
    ]))

    elements.append(overall_table)
    elements.append(PageBreak())  # End of the section with a page break

def generate_pdf_report(filename, df, max_week):
    performance_data, overall_data, overall_accuracy = analyze_weekly_betting_performance(df, max_week)

    doc = SimpleDocTemplate(filename, pagesize=letter)
    elements = []

    styles = getSampleStyleSheet()

    # Add the title page
    create_title_page(elements, max_week, 2024, "Noah Barulic")

    # Add the model performance page(s)
    create_model_performance_section(elements, performance_data, overall_data, overall_accuracy, styles)

    week_data = df[df['Week'] == max_week]

    elements.extend(generate_best_bets_visual(df, week_data))
    elements.append(PageBreak())

    # Loop through games for the selected week and generate pregame analysis
    for _, row in week_data.iterrows():
        home_team = row['Home Team']
        away_team = row['Away Team']
        home_info = team_data[team_data['Team'] == home_team].iloc[0].to_dict()
        away_info = team_data[team_data['Team'] == away_team].iloc[0].to_dict()
        # Extract scores
        avg_home_score = row['Avg_Home_Score']
        avg_away_score = row['Avg_Away_Score']

        vegas_spread, vegas_favored_team = parse_spread(row['Vegas_Spread'])
        predicted_spread, predicted_favored_team = parse_spread(row['Predicted Spread'])

        home_score = row.get('Actual_Home_Score', 'N/A')
        away_score = row.get('Actual_Away_Score', 'N/A')

        # Extract Vegas total and Over/Under
        vegas_total = row['Vegas_Total']

        # Calculate performance adjustment for both teams
        home_offensive_adjustment, home_defensive_adjustment = calculate_adjusted_score(df, home_team, row['Week'], is_home=True)
        away_offensive_adjustment, away_defensive_adjustment = calculate_adjusted_score(df, away_team, row['Week'], is_home=False)

        # Adjust the predicted scores based on past performance
        adjusted_home_score = avg_home_score * home_offensive_adjustment * away_defensive_adjustment
        adjusted_away_score = avg_away_score * away_offensive_adjustment * home_defensive_adjustment

        # Pregame analysis section for each game
        graphic_path = generate_matchup_graphic(home_info, away_info, round(adjusted_home_score), round(adjusted_away_score), vegas_favored_team, vegas_spread, vegas_total)
        elements.append(Image(graphic_path, width=6*inch, height=1.5*inch))

        # Generate performance graphs for the teams
        performance_graphs = generate_team_performance_graphs(df, home_info, away_info, max_week)

        # Load the images into Image flowables (adjust the size if necessary)
        home_combined_image = Image(performance_graphs['home_combined'], width=250, height=300)  # Adjusted size for one chart
        away_combined_image = Image(performance_graphs['away_combined'], width=250, height=300)

        # Create a 1x2 grid using a Table for the two combined charts
        performance_table = Table(
            [[home_combined_image, away_combined_image]],  # Two charts in one row
            colWidths=[3.5 * inch, 3.5 * inch]  # Adjust column widths
        )

        # Add some basic styling to the table for spacing and alignment
        performance_table.setStyle(TableStyle([
            ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
            ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
            ('LEFTPADDING', (0, 0), (-1, -1), 10),
            ('RIGHTPADDING', (0, 0), (-1, -1), 10),
            ('BOTTOMPADDING', (0, 0), (-1, -1), 10),
            ('TOPPADDING', (0, 0), (-1, -1), 10),
            ('GRID', (0, 0), (-1, -1), 0.5, colors.grey)  # Optional border for clarity
        ]))

        # Add the table of charts to the elements list
        elements.append(performance_table)
        elements.append(Spacer(1, 0.125 * inch))

        # Calculate win probabilities (just as an example, you would have your own logic for this)
        if predicted_favored_team == home_team:
            adjusted_predicted_spread = adjusted_home_score - adjusted_away_score
        else:
            adjusted_predicted_spread = adjusted_away_score - adjusted_home_score
        #print(f"Adj Spread: {adjusted_predicted_spread}; Favored team: {predicted_favored_team}")
        favored_win_prob = calculate_win_probability(adjusted_predicted_spread)  # Assume this function exists
        dog_win_prob = 1 - favored_win_prob  # Complementary probability

        if predicted_favored_team == home_team:
            home_win_prob = favored_win_prob
            away_win_prob = dog_win_prob
        else:
            home_win_prob = dog_win_prob
            away_win_prob = favored_win_prob

        # Generate the football field win probability graphic
        win_prob_graphic_path = generate_donut_chart_win_probability(home_info, away_info, home_win_prob, away_win_prob)

        # Add the win probability graphic to the report
        elements.append(Image(win_prob_graphic_path, width=4*inch, height=2.75*inch))

        # Add a page break after the charts
        elements.append(PageBreak())

    # Build the PDF
    doc.build(elements)

# Function to calculate win probability based on point spread
def calculate_win_probability(spread):
    # Coefficients for the logistic regression
    beta_0 = 0.15
    beta_1 = 0.13

    # Logistic regression formula for win probability
    win_prob = 1 / (1 + np.exp(-(beta_0 + beta_1 * spread)))

    return win_prob

def generate_donut_chart_win_probability(home_team_info, away_team_info, home_win_prob, away_win_prob):
    """
    Generates a donut chart for win probability with team logos in the center and percentage text on the sides.

    :param home_team_info: Dictionary containing the home team's information.
    :param away_team_info: Dictionary containing information for the away team.
    :param home_win_prob: Home team's win probability (0-1 scale).
    :param away_win_prob: Away team's win probability (0-1 scale).
    :return: Path to the saved image.
    """

    # Data for the donut chart
    sizes = [home_win_prob * 100, away_win_prob * 100]
    colors = [home_team_info['Color'], away_team_info['Color']]

    # Create a larger donut chart with thinner walls
    fig, ax = plt.subplots(figsize=(6, 6), dpi=150)
    wedges, _ = ax.pie(sizes, colors=colors, startangle=90, wedgeprops=dict(width=0.3, edgecolor='white'))

    # Add a smaller circle at the center to ensure it doesn't cover the logos
    center_circle = Circle((0, 0), 0.4, fc='white')  # Reduced size to 0.4
    ax.add_artist(center_circle)

    # Add a thin vertical line down the center to separate the two logos
    ax.plot([0, 0], [-0.55, 0.55], color='gray', linewidth=1, zorder=5)  # Shorter, lighter gray vertical line

    # Add the home and away percentages outside the donut
    ax.text(-1.7, 0, f"{home_win_prob*100:.1f}%", fontsize=16, fontweight='bold', color=home_team_info['Color'], ha='center', va='center')
    ax.text(1.7, 0, f"{away_win_prob*100:.1f}%", fontsize=16, fontweight='bold', color=away_team_info['Color'], ha='center', va='center')

    # Download and resize the logos
    try:
        home_logo = PILImage.open(BytesIO(requests.get(home_team_info['Logos[0]']).content)).resize((80, 80))
        away_logo = PILImage.open(BytesIO(requests.get(away_team_info['Logos[0]']).content)).resize((80, 80))
    except Exception as e:
        print(f"Error loading logos: {e}")
        home_logo = PILImage.new("RGB", (80, 80), color='gray')
        away_logo = PILImage.new("RGB", (80, 80), color='gray')

    # Adjust the logo positions to make them more centered and further apart
    logo_offset = 0.5  # Adjust this value to control how far apart the logos are

    # Place the home logo inside the donut on the left side, slightly higher
    ax.imshow(home_logo, extent=[-0.1 - logo_offset, 0.3 - logo_offset, -0.2, 0.2], zorder=10)

    # Place the away logo inside the donut on the right side, slightly higher
    ax.imshow(away_logo, extent=[-0.3 + logo_offset, 0.1 + logo_offset, -0.2, 0.2], zorder=10)

    # Set limits to ensure proper sizing and remove axis
    ax.set_xlim(-2, 2)
    ax.set_ylim(-1.5, 1.5)
    ax.axis('off')

    # Save the image
    image_path = f"{home_team_info['Abbreviation']}_vs_{away_team_info['Abbreviation']}_donut_chart.png"
    plt.savefig(image_path, bbox_inches='tight', dpi=150)
    plt.close()

    return image_path

# Function to clamp the adjusted score within the given range
def clamp_score(adjusted_score, original_score):
    lower_bound = original_score * 0.5
    upper_bound = original_score * 2.0
    return max(lower_bound, min(adjusted_score, upper_bound))

def calculate_adjusted_spreads_and_totals(df, week_data):
    best_bets_spread = []
    best_bets_total = []

    for _, row in week_data.iterrows():
        home_team = row['Home Team']
        away_team = row['Away Team']

        # Parse vegas_spread and get both the spread and the favored team
        vegas_spread, vegas_favored_team = parse_spread(row['Vegas_Spread'])

        # Convert vegas_total to float
        try:
            vegas_total = float(row['Vegas_Total'])
        except ValueError:
            vegas_total = 0.0  # Default to 0 if conversion fails

        # Calculate adjusted scores
        home_offensive_adjustment, home_defensive_adjustment = calculate_adjusted_score(df, home_team, row['Week'], is_home=True)
        away_offensive_adjustment, away_defensive_adjustment = calculate_adjusted_score(df, away_team, row['Week'], is_home=False)

        # Adjust the predicted scores
        avg_home_score = row['Avg_Home_Score']
        avg_away_score = row['Avg_Away_Score']

        # Calculate the adjusted home and away scores
        adjusted_home_score = avg_home_score * home_offensive_adjustment * away_defensive_adjustment
        adjusted_away_score = avg_away_score * away_offensive_adjustment * home_defensive_adjustment

        # Apply the clamping logic to ensure the adjusted score is within 0.5x to 2x of the original score
        adjusted_home_score = clamp_score(adjusted_home_score, avg_home_score)
        adjusted_away_score = clamp_score(adjusted_away_score, avg_away_score)

        # Calculate the adjusted spread
        if vegas_favored_team == home_team:
            adjusted_spread = adjusted_home_score - adjusted_away_score
        elif vegas_favored_team == away_team:
            adjusted_spread = adjusted_away_score - adjusted_home_score
        else:
            adjusted_spread = 0.0  # If we can't determine the favored team

        # Determine the model_favored_team based on the adjusted spread
        if adjusted_spread > vegas_spread:
            model_favored_team = vegas_favored_team  # Model agrees with Vegas
        elif adjusted_spread < vegas_spread:
            model_favored_team = home_team if vegas_favored_team == away_team else away_team  # Model picks the opposite team
        else:
            model_favored_team = "Pick'em"  # If the adjusted spread is 0, no team is favored

        # Calculate the adjusted total
        adjusted_total = adjusted_home_score + adjusted_away_score

        # Calculate the differences between the adjusted predictions and the Vegas numbers
        spread_diff = abs(adjusted_spread - vegas_spread)
        total_diff = abs(adjusted_total - vegas_total)

        # Save the data for each game
        best_bets_spread.append({
            'home_team': home_team,
            'away_team': away_team,
            'adjusted_spread': adjusted_spread,
            'vegas_spread': vegas_spread,  # Store the spread number
            'vegas_favored_team': vegas_favored_team,  # Store the Vegas favored team
            'model_favored_team': model_favored_team,  # Store the model favored team
            'spread_diff': spread_diff,
            'adjusted_total': adjusted_total,
            'vegas_total': vegas_total,
            'total_diff': total_diff
        })

        best_bets_total.append({
            'home_team': home_team,
            'away_team': away_team,
            'adjusted_total': adjusted_total,
            'vegas_total': vegas_total,
            'total_diff': total_diff
        })

    return pd.DataFrame(best_bets_spread), pd.DataFrame(best_bets_total)

# Function to generate the top 15 best bets based on spread and total differences
def get_top_15_best_bets(df_best_bets, metric):
    # Sort by the largest difference for the specified metric (spread_diff or total_diff)
    df_best_bets = df_best_bets.sort_values(by=[metric], ascending=False)

    # Get the top 15 best bets
    top_15_bets = df_best_bets.head(15)

    return top_15_bets

def generate_best_bets_visual(df, week_data):
    # Calculate the adjusted spreads and totals for all games
    df_best_bets_spread, df_best_bets_total = calculate_adjusted_spreads_and_totals(df, week_data)

    # Get the top 15 bets for spread and total
    top_15_spread_bets = get_top_15_best_bets(df_best_bets_spread, 'spread_diff')
    top_15_total_bets = get_top_15_best_bets(df_best_bets_total, 'total_diff')

    # Define headers for the spread bets table (new "Pick" column, and updated spread columns)
    spread_headers = ['Matchup', 'Predicted Spread', 'Vegas Spread', 'Pick']

    # Helper function to get spread display format
    def format_spread(team, spread):
        return f"{team} -{abs(spread):.1f}"

    # Helper function to generate the "Pick" symbol for the Spread
    def get_team_logo(team_name, team_data):
        """
        Fetch the logo URL for a given team from the team_data DataFrame.
        If the team is not found, return a placeholder.
        """
        try:
            logo_url = team_data.loc[team_data['Team'] == team_name, 'Logos[0]'].values[0]
            logo = Image(logo_url, width=20, height=20)  # Adjust the size of the logo as needed
            return logo
        except IndexError:
            # Return a default placeholder logo if team logo is not found
            return Paragraph("No Logo", getSampleStyleSheet()['BodyText'])

    # Table data for spread bets with formatted spreads and a "Pick" column (logo in place of pick text)
    spread_bets_table_data = [spread_headers] + [
        [
            f"{row['home_team']} vs {row['away_team']}",
            f"{row['model_favored_team']} -{abs(row['adjusted_spread']):.1f}",  # Adjusted Spread
            f"{row['vegas_favored_team']} -{abs(row['vegas_spread']):.1f}",  # Vegas Spread with team
            get_team_logo(row['model_favored_team'], team_data)  # Display the model's favored team logo in the Pick column
        ] for _, row in top_15_spread_bets.iterrows()
    ]

    # Define headers for the total bets table (as before)
    total_headers = ['Matchup', 'Predicted Total', 'Vegas Total', 'Pick']

    # Helper function for total pick (already done in previous implementation)
    def get_total_pick_symbol(adjusted_total, vegas_total):
        if adjusted_total > vegas_total:
            return '<font color="green">▲</font>'  # Green upward triangle
        else:
            return '<font color="red">▼</font>'  # Red downward triangle

    # Create a custom style for centered text
    centered_style = ParagraphStyle(
        'Centered',
        parent=getSampleStyleSheet()['BodyText'],
        alignment=1  # 1 means centered
    )

    # Table data for total bets with a "Pick" column
    total_bets_table_data = [total_headers] + [
        [
            f"{row['home_team']} vs {row['away_team']}",
            f"{round(row['adjusted_total'])}",  # Predicted Total (rounded)
            f"{row['vegas_total']:.1f}",  # Vegas Total (1 decimal place)
            Paragraph(get_total_pick_symbol(row['adjusted_total'], row['vegas_total']), centered_style)  # Use the centered style
        ] for _, row in top_15_total_bets.iterrows()
    ]

    # Adjust column widths to prevent spilling over edges
    spread_col_widths = [2.5 * inch, 1.75 * inch, 1.75 * inch, 1.0 * inch]  # Adjust to make the "Pick" column smaller
    total_col_widths = [3.0 * inch, 1.5 * inch, 1.5 * inch, 1.0 * inch]   # Adjust for total bets, with narrower Pick column

    # Center-align the Pick symbols (triangles) in total bets
    table_style_total = TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.grey),  # Header row background
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),  # Header text color
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),  # Center-align all cells
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),  # Bold header font
        ('BACKGROUND', (0, 1), (-1, -1), colors.whitesmoke),  # White background for rows
        ('TEXTCOLOR', (0, 1), (-1, -1), colors.black),  # Black text for data rows
        ('GRID', (0, 0), (-1, -1), 0.5, colors.black),  # Grid lines between cells
        ('FONTSIZE', (0, 0), (-1, -1), 10),  # Font size for all cells
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),  # Padding for header row
        ('BOTTOMPADDING', (0, 1), (-1, -1), 10),  # Padding for body rows
        ('ALIGN', (3, 1), (3, -1), 'CENTER')  # Center-align "Pick" column
    ])

    # Create the table for spread bets
    spread_bets_table = Table(spread_bets_table_data, colWidths=spread_col_widths)
    spread_bets_table.setStyle(table_style_total)  # Reuse style

    # Create the table for total bets
    total_bets_table = Table(total_bets_table_data, colWidths=total_col_widths)
    total_bets_table.setStyle(table_style_total)

    # Create elements for both tables and insert PageBreak between them
    elements = []
    elements.append(Paragraph("Top 15 Best Spread Bets", getSampleStyleSheet()['Heading2']))
    elements.append(spread_bets_table)
    elements.append(PageBreak())  # Insert a page break before the next table

    elements.append(Paragraph("Top 15 Best Total Bets", getSampleStyleSheet()['Heading2']))
    elements.append(total_bets_table)

    return elements

def main(df):
    while True:
        print("\nChoose an option:")
        print("1. Input Vegas spread and total")
        print("2. Input actual scores")
        print("3. Generate Pregame Betting Analysis Report")
        print("4. Exit")

        choice = input("Enter the number of your choice (1, 2, 3, or 4): ")

        if choice == '1':
            week = int(input("Enter the week number you want to work on: "))
            input_vegas_spread(df, week)
        elif choice == '2':
            week = int(input("Enter the week number you want to work on: "))
            input_actual_scores(df, week)
        elif choice == '3':
            week = int(input("Enter the week number you want to generate the report for: "))
            generate_pdf_report("betting_analysis_report.pdf", df_projections, week)
        elif choice == '4':
            break
        else:
            print("Invalid choice. Please enter 1, 2, 3, or 4.")

def generate_matchup_graphic(home_team_info, away_team_info, predicted_home_score, predicted_away_score, vegas_favored_team, vegas_spread, vegas_total):
    """
    Generates a clean visual representation of the game matchup.
    Includes logos, predicted scores, and Vegas line/total below.
    """

    # Extract relevant information from both teams
    home_team_abbreviation = home_team_info['Abbreviation']
    home_team_color = home_team_info['Color']
    home_team_logo_url = home_team_info['Logos[0]']
    away_team_abbreviation = away_team_info['Abbreviation']
    away_team_color = away_team_info['Color']
    away_team_logo_url = away_team_info['Logos[0]']

    # Download team logos
    try:
        home_team_logo = PILImage.open(BytesIO(requests.get(home_team_logo_url).content))
        away_team_logo = PILImage.open(BytesIO(requests.get(away_team_logo_url).content))
    except Exception as e:
        home_team_logo = PILImage.new("RGB", (100, 100), color=home_team_color)
        away_team_logo = PILImage.new("RGB", (100, 100), color=away_team_color)

    # Setup figure (6 x 1.8 inches)
    fig, ax = plt.subplots(figsize=(6, 1.8))
    ax.axis('off')

    # Place logos without stretching
    ax.imshow(home_team_logo, aspect='auto', extent=[0.1, 0.3, 0.2, 0.8])
    ax.imshow(away_team_logo, aspect='auto', extent=[0.7, 0.9, 0.2, 0.8])

    # Add abbreviations and bolder predicted scores
    ax.text(0.2, 0.1, home_team_abbreviation, fontsize=12, ha='center', color='black')
    ax.text(0.8, 0.1, away_team_abbreviation, fontsize=12, ha='center', color='black')
    ax.text(0.45, 0.5, str(predicted_home_score), fontsize=16, fontweight='bold', ha='right', color=home_team_color)
    ax.text(0.55, 0.5, str(predicted_away_score), fontsize=16, fontweight='bold', ha='left', color=away_team_color)
    ax.text(0.5, 0.5, "-", fontsize=16, ha='center', color='black')

    # Thin black dash closer to scores
    ax.plot([0.4, 0.6], [0.4, 0.4], color='black', lw=4)

    # Vegas info below with more compact width
    vegas_text = f"Vegas Line: {vegas_favored_team} -{vegas_spread}\nVegas Total: {vegas_total}"
    ax.text(0.5, 0.2, vegas_text, fontsize=8, ha='center', color='black', wrap=True)

    # Save the image
    image_path = f"{home_team_abbreviation}_vs_{away_team_abbreviation}_matchup.png"
    plt.savefig(image_path, bbox_inches='tight', dpi=150)
    plt.close()

    return image_path

In [26]:
3# Define the path to your CSV file in Google Drive
file_path = '/content/drive/MyDrive/CFB_Model/updated_projections.csv'

# Load the CSV file into a DataFrame
df_projections = pd.read_csv(file_path)

# Call the main function
main(df_projections)

# Save the updated dataset
df_projections.to_csv("/content/drive/MyDrive/CFB_Model/updated_projections.csv", index=False)


Choose an option:
1. Input Vegas spread and total
2. Input actual scores
3. Generate Pregame Betting Analysis Report
4. Exit
Enter the number of your choice (1, 2, 3, or 4): 3
Enter the week number you want to generate the report for: 5

Choose an option:
1. Input Vegas spread and total
2. Input actual scores
3. Generate Pregame Betting Analysis Report
4. Exit
Enter the number of your choice (1, 2, 3, or 4): 4
